In [1]:

# data manipulation
import pandas as pd

# plotly 
import plotly.express as px
import plotly.graph_objects as go
# dashboards
import dash 
from jupyter_dash import JupyterDash #for running the dashboard in jupyter notebook
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Output, Input
import dash_bootstrap_components as dbc #for dashboard themes

import numpy as np
import warnings
warnings.filterwarnings('ignore')


<ipython-input-1-1f663c3dc0e1>:10: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
<ipython-input-1-1f663c3dc0e1>:11: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [2]:
#calculate the cumulative distance for each team for each season
def calc_cd(data,df,team_value):
    #sort the games by date ascending
    d=data[data['team']==team_value].sort_values(by='datetime', key=pd.to_datetime)
    for season_value in d['season'].unique():
        dd=d[d['season']==season_value]
        dd['cd']=np.cumsum(dd['distance_traveled'])
        for row in dd.index:
            df.loc[row]=dd.loc[row]
            df.loc[row,'cd']=dd.loc[row,'cd']
    return df

In [18]:
#export cumulative dist in each season for each team to schedule_cd.csv
data = pd.read_csv('schedule.csv')
data['month']=pd.to_datetime(data.datetime).dt.month
data['week']=pd.to_datetime(data.datetime).dt.weekofyear
l=list(data.columns)
l.append('cd')
l.append('acd')
df=pd.DataFrame(columns=l, index=range(0,data.shape[0]))
for team_value in data['team'].unique():
    df=calc_cd(data,df,team_value)
df.to_csv('schedule_cd.csv')
df.team.unique()

array(['POR', 'LAC', 'DEN', 'HOU', 'DAL', 'NOH', 'DET', 'CHI', 'PHO',
       'SAC', 'NJN', 'TOR', 'IND', 'PHI', 'WAS', 'MIL', 'MIN', 'NYK',
       'MIA', 'ORL', 'MEM', 'BOS', 'GSW', 'UTA', 'SAS', 'CLE', 'CHA',
       'OKC', 'ATL', 'LAL', 'BKN', 'NOP', 'CHO'], dtype=object)

In [3]:
df = pd.read_csv('schedule_cd.csv',index_col=0)
df.head()

,boxscore_index,date,datetime,game,location,distance_traveled,losses,opponent_abbr,opponent_name,opponent_long,...,season,opp_coords,team_coords,back_to_back,3_in_4,4_in_5,month,week,cd,acd
0,200810280LAL,"Tue, Oct 28, 2008",10/28/08,1,Away,826.133254,1.0,LAL,Los Angeles Lakers,-118.267222,...,2009,"(34.043056, -118.267222)","(45.531667, -122.666667)",0,0,0,10,44,826.133254,NaN
1,200810310POR,"Fri, Oct 31, 2008",10/31/08,2,Home,826.133254,1.0,SAS,San Antonio Spurs,-98.437500,...,2009,"(29.426944, -98.4375)","(45.531667, -122.666667)",0,0,0,10,44,1652.266508,NaN
2,200811010PHO,"Sat, Nov 1, 2008",11/1/08,3,Away,1005.964440,2.0,PHO,Phoenix Suns,-112.071389,...,2009,"(33.445833, -112.071389)","(45.531667, -122.666667)",1,0,0,11,44,2658.230948,NaN
3,200811050UTA,"Wed, Nov 5, 2008",11/5/08,4,Away,505.045543,3.0,UTA,Utah Jazz,-111.901111,...,2009,"(40.768333, -111.901111)","(45.531667, -122.666667)",0,0,0,11,45,3163.276491,NaN
4,200811060POR,"Thu, Nov 6, 2008",11/6/08,5,Home,634.959453,3.0,HOU,Houston Rockets,-95.362222,...,2009,"(29.750833, -95.362222)","(45.531667, -122.666667)",1,0,0,11,45,3798.235944,NaN


In [13]:
df = pd.read_csv('schedule_acd.csv',index_col=0)
df.head()

,boxscore_index,date,datetime,game,location,distance_traveled,losses,opponent_abbr,opponent_name,opponent_long,...,season,opp_coords,team_coords,back_to_back,3_in_4,4_in_5,month,week,cd,acd
0,200810280LAL,"Tue, Oct 28, 2008",10/28/08,1,Away,826.133254,1.0,LAL,Los Angeles Lakers,-118.267222,...,2009,"(34.043056, -118.267222)","(45.531667, -122.666667)",0,0,0,10,44,826.133254,1176.746341
1,200810310POR,"Fri, Oct 31, 2008",10/31/08,2,Home,826.133254,1.0,SAS,San Antonio Spurs,-98.437500,...,2009,"(29.426944, -98.4375)","(45.531667, -122.666667)",0,0,0,10,44,1652.266508,1176.746341
2,200811010PHO,"Sat, Nov 1, 2008",11/1/08,3,Away,1005.964440,2.0,PHO,Phoenix Suns,-112.071389,...,2009,"(33.445833, -112.071389)","(45.531667, -122.666667)",1,0,0,11,44,2658.230948,1176.746341
3,200811050UTA,"Wed, Nov 5, 2008",11/5/08,4,Away,505.045543,3.0,UTA,Utah Jazz,-111.901111,...,2009,"(40.768333, -111.901111)","(45.531667, -122.666667)",0,0,0,11,45,3163.276491,3072.231073
4,200811060POR,"Thu, Nov 6, 2008",11/6/08,5,Home,634.959453,3.0,HOU,Houston Rockets,-95.362222,...,2009,"(29.750833, -95.362222)","(45.531667, -122.666667)",1,0,0,11,45,3798.235944,3072.231073


In [41]:
df=df[df['season']==2022]
gb = df.groupby('week')
a=gb.agg({'datetime' : np.min,
        'cd' : np.mean})
a.sort_values(by='datetime', key=pd.to_datetime, inplace=True)
a

,datetime,cd
week,,
42,10/19/21,1214.439632
43,10/25/21,2777.545470
44,11/1/21,4650.710464
45,11/10/21,6280.360539
46,11/15/21,7988.146676
47,11/22/21,9892.603087
48,11/29/21,11683.398191
49,12/10/21,13220.413360
50,12/13/21,15160.445341


In [43]:
fig = go.Figure()

df = pd.read_csv('schedule_acd.csv',index_col=0)
df2=df[df['season']==2022]
gb = df2.groupby('week')
a=gb.agg({'datetime' : np.min,
        'cd' : np.mean})
a.sort_values(by='datetime', key=pd.to_datetime, inplace=True)

fig.add_trace(
    go.Scatter(x = pd.to_datetime(a['datetime']), 
            y = a['cd'],
            mode = 'lines',
            line={'color': 'gray'},
            name='Average',
            )
)
fig.show()

In [45]:
team_value='ATL'
df = pd.read_csv('schedule_acd.csv',index_col=0)
df=df[df['season']==2022]
d=df[df['team']==team_value].sort_values(by='datetime', key=pd.to_datetime)

fig = go.Figure()
fig.add_trace(
    go.Scatter(x = pd.to_datetime(d['datetime']), 
            y = d['cd'],
            mode = 'lines',
            line={'color': 'green'},
            name=team_value,
            )
)
fig.show()

In [48]:
app = JupyterDash(__name__,external_stylesheets = [dbc.themes.BOOTSTRAP])

# read the data
data = pd.read_csv('schedule.csv')

# list of all unique teams
teams=data['opponent_name'].unique()

# list of all unique team abbreviation
team_abbr = data['opponent_abbr'].unique()

#list of all unique seasons
seasons=data['season'].unique()

app.layout = html.Div([
    
    # top left drop menu for teams
    html.Div([
            dcc.Dropdown(
                id='team_value',
                options=[{'label': teams[i], 'value': team_abbr[i]} for i in range(len(teams))],
                value='ATL'
            )
    ], style={'width': '30%', 'display': 'inline-block'}), 
   
    
    # top middle drop menu for seasons
    html.Div([
            dcc.Dropdown(
                id='season_value',
                options=[{'label': s, 'value': s} for s in seasons],
                value=2022
            )
    ], style={'width': '30%', 'float': 'center', 'display': 'inline-block'}),


    # top left date range picker 
    html.Div([
        dcc.DatePickerRange(
            id='datepicker',
            month_format='YYYY MMM'
        )
    ], style={'width': '30%', 'float': 'right', 'display': 'inline-block'}),

    dcc.Graph(id='cumdist-graphic')
])
@app.callback(
    Output('datepicker', 'start_date'),
    Output('datepicker', 'end_date'),
    Input('season_value', 'value')
)

def update_date_range(season_value): 
    df = pd.read_csv('schedule.csv', index_col=0)
    df = df[df['season']==season_value]
    df = df[df['playoffs']==0]
    min_date = df['datetime'].min()
    max_date = df['datetime'].max()
    return min_date, max_date

@app.callback(
    Output('cumdist-graphic', 'figure'),
    Input('team_value', 'value'),
    Input('season_value', 'value'),
    Input('datepicker', 'start_date'),
    Input('datepicker', 'end_date'))

def update_graph(team_value, season_value, start_date, end_date):
    
    fig = go.Figure()
    
    
    df = pd.read_csv('schedule_acd.csv',index_col=0)
    df = df[df['season']==season_value]
    df = df[(df.datetime > start_date) & (df.datetime < end_date)]
    
    #for average
    gb = df.groupby('week')
    a=gb.agg({'datetime' : np.min,
            'cd' : np.mean})
    a.sort_values(by='datetime', key=pd.to_datetime, inplace=True)

    #for team
    d=df[df['team']==team_value].sort_values(by='datetime', key=pd.to_datetime)

    fig.add_trace(
        go.Scatter(x = pd.to_datetime(a['datetime']), 
                y = a['cd'],
                mode = 'lines',
                line={'color': 'gray'},
                name='Average',
                )
    )

    fig.add_trace(
        go.Scatter(x = pd.to_datetime(d['datetime']), 
                y = d['cd'],
                mode = 'lines',
                line={'color': 'green'},
                name=team_value,
                )
    )
    
    return fig
    
if __name__ == '__main__':
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/
